In [ ]:
# Dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import scipy.stats as ss
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from scipy.stats import norm

import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, LeakyReLU, Dropout
from keras import backend
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.utils import get_custom_objects
from keras import backend as K
from keras.activations import elu, sigmoid
import tensorflow as tf

In [ ]:
tf.keras.backend.set_floatx('float32')

In [ ]:
N = norm.cdf
def BS_CALL(S, K, T, r, q , sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S*(math.exp(-q*T))* N(d1) - K * np.exp(-r*T)* N(d2)

In [ ]:
price = np.random.uniform(10, 20, 300000)
strike = np.random.uniform(10, 20, 300000)
time = np.random.uniform(0.1, 3, 300000)
intrest = np.random.uniform(0.01, 0.03, 300000)
divident = np.random.uniform(0, 0.03, 300000)
volatility = np.random.uniform(0.05, 0.3, 300000)
call = np.zeros(300000)

In [ ]:
for i in range(0,300000):
  call[i] = BS_CALL(price[i],strike[i],time[i],intrest[i],divident[i],volatility[i])

In [ ]:
df = pd.DataFrame()

In [ ]:
df['Stock'] = price.tolist()
df['Strike'] = strike.tolist()
df['Time'] = time.tolist()
df['Intrest'] = intrest.tolist()
df['Divident'] = divident.tolist()
df['Volatility'] = volatility.tolist()
df['Call'] = call.tolist()

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(({
    'Stock': df['Stock'].values,
    'Strike': df['Strike'].values,
    'Time': df['Time'].values.astype(np.float32),  # Make sure Time is a float32 tensor for gradient
    'Intrest': df['Intrest'].values,
    'Divident': df['Divident'].values,
    'Volatility': df['Volatility'].values
}, df['Call'].values))

In [ ]:
df_filtered = df[df['Call'] <= 10]
df_filtered = df_filtered[df_filtered['Call'] >= 0.001]

In [ ]:
df_filtered['Stock'] = df_filtered['Stock'].div(df_filtered.Strike)
df_filtered['Call'] = df_filtered['Call'].div(df_filtered.Strike)
MinCall = df_filtered.loc[df_filtered["Call"].idxmin()].Call
df_filtered['Strike'] = 1
Strike = df_filtered.Strike.values
df_filtered = df_filtered.drop('Strike', axis=1)

In [ ]:
df_filtered['Call'] = df_filtered['Call'] - 0.5

In [ ]:
prices = df_filtered.values

In [ ]:
X = prices[:,:5]
y = prices[:,5:6]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def custom_activation(x):
    return (K.softplus(x)-0.5)

get_custom_objects().update({'custom_activation': Activation(custom_activation)})

In [ ]:
def MELU(z):
  cond = K.greater(z, 0)
  return K.switch(cond, (((0.5*(K.pow((z),2)))+(0.02*(z)))/((z)+0.040816326)),0.49*(K.exp(z)-1))

get_custom_objects().update({'MELU': Activation(MELU)})

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
tfe = tf.compat.v1.estimator.eager
tf.enable_eager_execution()

AttributeError: module 'tensorflow_estimator.python.estimator.api._v1.estimator' has no attribute 'eager'

In [ ]:
from pandas.core.arrays.arrow.array import Any
nodes = 128
def make_model():
  net = tf.keras.Sequential()

# Layer 1
  net.add(tf.keras.layers.Dense(nodes , input_shape=(5,)))
  net.add(tf.keras.layers.Activation(LeakyReLU(1.0)))

# Layer 2
  net.add(tf.keras.layers.Dense(nodes))
  net.add(tf.keras.layers.Activation (MELU))

# Layer 3
  net.add(tf.keras.layers.Dense(nodes))
  net.add(tf.keras.layers.Activation (MELU))

# Layer 4 - Output Layer
  net.add(tf.keras.layers.Dense(1))
  net.add(tf.keras.layers.Activation(custom_activation))

In [ ]:
def compute_loss(pred, actual):
    return tf.reduce_mean(tf.square(tf.subtract(pred, actual)))


In [ ]:
def compute_gradient(model, pred, actual):
    """compute gradients with given noise and input"""
    with tf.GradientTape() as tape:
        loss = compute_loss(pred, actual)
    grads = tape.gradient(loss, model.variables)
    return grads, loss

In [ ]:
def apply_gradients(optimizer, grads, model_vars):
    optimizer.apply_gradients(zip(grads, model_vars))

In [ ]:
model = make_model()
optimizer = tf.train.AdamOptimizer(1e-4)

In [ ]:
# def custom_loss_pass(model, x_tensor):
#     def custom_loss(y_true,y_pred):
#         with tf.GradientTape() as t:
#             t.watch(x_tensor)
#             output = model(x_tensor)
#         DyDX = t.gradient(output, x_tensor)
#         dy_t = DyDX[:, 5:6]
#         R_pred=dy_t
#         # loss_data = tf.reduce_mean(tf.square(yTrue - yPred), axis=-1)
#         loss_PDE = tf.reduce_mean(tf.square(R_pred))
#         return loss_PDE
#     return custom_loss

In [ ]:
# def custom_penalty(x, lambda_val):
#     return tf.where(x >= 0, lambda_val * tf.pow(x, 4), tf.zeros_like(x))


In [ ]:
# Assume a simple model
# model = tf.keras.Sequential([...])
# # Assume lambda values for each term
# lambda_1 = 0.01  # example value for lambda_1
# lambda_2 = 0.01  # example value for lambda_2
# lambda_3 = 0.01  # example value for lambda_3
# optimizer='adam'
# # Define the constant K
# K = tf.constant([...], dtype=tf.float32)

# # Training loop
# for epoch in range(15):
#     for step, (X_train,y_train) in enumerate(train_dataset):
#         # T is part of x_batch_train and is a trainable variable
#         with tf.GradientTape(persistent=True) as tape:
#             # Ensure the tape is watching the trainable variables
#             tape.watch(model.trainable_variables)

#             # Forward pass: compute predictions
#             predictions = model(X_train)
#             # Compute the base loss (MSE)
#             base_loss = tf.keras.losses.mean_squared_error(y_train, predictions)

#             # Compute the derivative with respect to T
#             grads_wrt_T = tape.gradient(predictions, [T])[0]

#             # Apply the penalty functions
#             penalty_2 = custom_penalty(grads_wrt_T, lambda_2)

#             # Since the second derivative of C_ANN with respect to K is zero (K is a constant),
#             # penalty_1 will be zero and does not need to be computed.
#             # penalty_3 is not applied because there is no gradient with respect to a constant K.

#             # Sum up penalties
#             penalties = tf.reduce_sum(penalty_2)

#             # Total loss is base loss plus penalties
#             total_loss = base_loss + penalties

#         # Compute gradients of total loss with respect to model's trainable weights
#         gradients = tape.gradient(total_loss, model.trainable_variables)

#         # Update weights
#         optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# # Clean up the persistent tape
# del tape


In [ ]:
batched_dataset = dataset.batch(64)

In [ ]:
model.compile(loss= tf.keras.losses.MeanSquaredError(),
  optimizer='adam',)

In [ ]:
for features, _ in batched_dataset:
    with tf.GradientTape() as tape:
        # Extract the 'Time' tensor and ensure it's being watched by the gradient tape
        time_tensor = features['Time']
        tape.watch(time_tensor)

        # Reshape your features into the correct format expected by the model
        # This could be a single concatenated tensor or a list of tensors
        model_input = tf.stack([features['Stock'], features['Strike'], time_tensor,features['Intrest'], features['Divident'], features['Volatility']], axis=-1)

        # Run the model on this batch of features to get predictions
        predictions = model(model_input, training=True)  # Set training to False if not during training

        # Compute the gradient of the predictions with respect to 'Time'
        grads = tape.gradient(predictions, time_tensor)
        grads_list.append(grads)

InvalidArgumentError: cannot compute Pack as input #2(zero-based) was expected to be a double tensor but is a float tensor [Op:Pack] name: stack

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               768       
                                                                 
 activation_2 (Activation)   (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 activation_3 (Activation)   (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 activation_4 (Activation)   (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [ ]:
model.fit(X_train, y_train, epochs = 5, batch_size = 64)

Epoch 1/5


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "<ipython-input-17-c18ab278d180>", line 4, in custom_loss  *
        t.watch(x_tensor)

    ValueError: Passed in object [[1.46421546 0.42758729 0.02569136 0.01189804 0.24322787]
     [0.93022739 2.7911211  0.01560296 0.02890846 0.07344184]
     [1.42618641 0.25738741 0.01524909 0.00998639 0.05472978]
     ...
     [0.67100654 0.91386802 0.01685517 0.00366419 0.24841322]
     [0.91249979 0.5830864  0.01691423 0.02982428 0.14258801]
     [1.36199063 1.73544565 0.01216461 0.00358919 0.24197907]] of type 'ndarray', not tf.Tensor or tf.Variable or ExtensionType.


In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
plt.figure(figsize = (15,10))
plt.scatter((y_test+0.5)*10, (predictions+0.5)*10)
plt.axis((0, 10.5, 0, 10.5))
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.plot([0,10], [0,10], 'r')
plt.grid(True)

In [ ]:
InSamplePredictions = model.predict(X_train)

In [ ]:
#in sample (80)
plt.figure(figsize = (15,10))
plt.scatter((y_train+0.5)*10, (InSamplePredictions+0.5)*10)
plt.axis((0, 10.5, 0, 10.5))
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.plot([0,10], [0,10], 'r')
plt.grid(True)